<a href="https://colab.research.google.com/github/masfana016/OpenAIs_Agent_SDK/blob/main/zoom_mail_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 736.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.5/168.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
gemini_api_key = userdata.get("GOOGLE_API_KEY")

In [ ]:
from agents import set_tracing_export_api_key, Agent, Runner

tracing_api_key = userdata.get("OPENAI_API_KEY")
set_tracing_export_api_key(tracing_api_key)

In [ ]:
from agents import AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig

client = AsyncOpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/",)

model = OpenAIChatCompletionsModel(model="gemini-2.5-flash", openai_client=client)

config = RunConfig(model = model,
                   model_provider = client,
                   tracing_disabled = False
                   )

In [ ]:
from agents import function_tool
import requests
import json
from datetime import datetime
import pytz

# Define timezone for Pakistan
PKT = pytz.timezone('Asia/Karachi')


CLIENT_ID = userdata.get("ZOOM_CLIENT_ID")
CLIENT_SECRET = userdata.get("ZOOM_CLIENT_SECRET")
ACCOUNT_ID = userdata.get("ZOOM_ACCOUNT_ID")

def get_access_token():
    url = "https://zoom.us/oauth/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "account_credentials",
        "account_id": ACCOUNT_ID
    }
    auth = (CLIENT_ID, CLIENT_SECRET)

    response = requests.post(url, headers=headers, data=payload, auth=auth)
    try:
        return response.json()["access_token"]
    except KeyError:
        return {
            "error": "Failed to fetch token",
            "status_code": response.status_code,
            "response": response.text
        }

def getUsers():
    """Fetch Zoom user information"""
    token = get_access_token()
    if isinstance(token, dict) and "error" in token:
        return token

    headers = {
        'authorization': f'Bearer {token}',
        'content-type': 'application/json'
    }
    response = requests.get('https://api.zoom.us/v2/users/', headers=headers)
    return response.json()

def getMeetingParticipants(meeting_id):
    """Fetch participants of a live Zoom meeting by meeting_id"""
    token = get_access_token()
    if isinstance(token, dict) and "error" in token:
        return token

    headers = {
        'authorization': f'Bearer {token}',
        'content-type': 'application/json'
    }
    response = requests.get(
        f'https://api.zoom.us/v2/metrics/meetings/{meeting_id}/participants',
        headers=headers
    )
    return response.json()

def convert_appointment_time_to_zoom_format(appointment_time):
    """Convert the appointment time from 'DD-MM-YYYY HH:MM:SS' to 'YYYY-MM-DDTHH:MM:SS' format"""
    # Parse the appointment time string to a datetime object
    appointment_dt = datetime.strptime(appointment_time, "%d-%m-%Y %H:%M:%S")

    # Convert it to Pakistan's timezone
    appointment_dt = PKT.localize(appointment_dt)

    # Convert the datetime object to the desired string format
    return appointment_dt.strftime("%Y-%m-%dT%H:%M:%S")

meetingdetails = {
    "topic": "Telemedicine",
    "type": 2,
    "start_time": "14-05-2025 10:21:57",
    "duration": "45",
    "timezone": "Asia/Karachi",
    "agenda": "test",
    "recurrence": {
        "type": 1,
        "repeat_interval": 1
    },
    "settings": {
        "host_video": True,
        "participant_video": True,
        "join_before_host": False,
        "mute_upon_entry": False,
        "watermark": True,
        "audio": "voip",
        "auto_recording": "cloud"
    }
}

@function_tool
def createMeeting():
    """Create a new Zoom meeting and return details"""
    token = get_access_token()
    if isinstance(token, dict) and "error" in token:
        return token

    headers = {
        'authorization': f'Bearer {token}',
        'content-type': 'application/json'
    }

    response = requests.post(
        'https://api.zoom.us/v2/users/me/meetings',
        headers=headers,
        data=json.dumps(meetingdetails)
    )

    if response.status_code == 201:
        return response.json()
    else:
        return {
            "error": "Failed to create meeting",
            "status_code": response.status_code,
            "response": response.text
        }

In [ ]:
from agents import function_tool
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

@function_tool
def send_email(to_email: str, subject: str, body: str):
    """
    Sends an email using Gmail's SMTP server.

    Parameters:
    to_email (str): Recipient email address.
    subject (str): Subject of the email.
    body (str): Body content of the email.

    Returns:
    None
    """

    email_address = userdata.get('EMAIL_ADDRESS')
    APP_PASSWORD = userdata.get('APP_PASSWORD')

    try:
        with smtplib.SMTP_SSL(host="smtp.gmail.com", port=465) as server:
            server.login(email_address, APP_PASSWORD)

            msg = MIMEMultipart()
            msg['From'] = email_address
            msg['To'] = to_email
            msg['Subject'] = subject
            msg.attach(MIMEText(body, 'plain'))

            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
from agents import Agent, Runner
import asyncio

email_agent = Agent(name="Email Assistant",
                      instructions=""" Be a helpfull Email assistant. Whenever ## **Sending Emails automatically creatig a zoom meeting details**:
                      - After successfully generate a zoom meeting detail, generate well-structured and contextually email bodies with those generated email details and send to the given email address.
                      Sending Emails**:
                      - Generate well-structured and contextually relevant email bodies based on user requests.
                      - Send emails using Gmail's SMTP server, utilizing the provided recipient email address, subject, and body content.
                      - If replying to an email, ensure the recipient's email address is correctly identified from the previously read emails.
                      - Support attaching files to outgoing emails if requested by the user.
                        """,
                      tools = [send_email],
                      handoff_description="Use this agent to send emails with the Zoom meeting details after a meeting is successfully created, or to handle any other email-related tasks such as replying or composing new messages."
                      )

zoom_agent = Agent(name="Zoom Assistant",
                      instructions=""" Be a helpfull zoom meeting assistant. Whenever user ask to create a zoom meeting.
                      **Zoom Meeting Creation**:
                        - The Zoom meeting is configured with the following details (as per the provided `meetingdetails`):
                          - Topic: "Telemedicine"
                          - Type: Scheduled meeting
                          - Start Time: Aligned with the appointment's day and time
                          - Duration: 45 minutes
                          - Timezone: Asia/Karachi
                          - Settings: Host and participant video enabled, watermark enabled, cloud auto-recording, etc.
                        - After the meeting is created, include the following in the confirmation message and email:
                          - Meeting ID
                          - Join URL (for participants)
                          - Password
                          - Start Time
                          - Host Start URL (if applicable)
                        - Ensure the meeting details are stored and retrievable when the user views their appointment details.
                        """,
                      tools = [createMeeting],
                      handoffs=[email_agent]
                      )

In [ ]:
from agents import Runner

async def main():

  while True:
    agent = zoom_agent
    result = await Runner.run(
        agent,
        input= input("How Can I help you with? "),
        run_config=config,
    )

    print(f"Response: {result.to_input_list()[-1]['content'][0]['text'].strip()}")

if __name__ == "__main__":
  asyncio.run(main())
